In [1]:
import os
import sys
if not os.getcwd().endswith("Submodular"):
    sys.path.append('../Submodular')    

In [2]:
import DeviceDir

DIR, RESULTS_DIR = DeviceDir.get_directory()
device, NUM_PROCESSORS = DeviceDir.get_device()

In [3]:
from ipynb.fs.full.Dataset import get_data
from ipynb.fs.full.Dataset import datasets as available_datasets
from ipynb.fs.full.Utils import save_plot

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_sparse import SparseTensor, matmul
# from torch_geometric.nn.models import MLP
from torch_geometric.nn.conv.gcn_conv import gcn_norm
import numpy as np
import scipy.sparse
from tqdm import tqdm

In [5]:
class MLP(nn.Module):
    """ adapted from https://github.com/CUAI/CorrectAndSmooth/blob/master/gen_models.py """
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout=.5):
        super(MLP, self).__init__()
        self.lins = nn.ModuleList()
        self.bns = nn.ModuleList()
        if num_layers == 1:
            # just linear layer i.e. logistic regression
            self.lins.append(nn.Linear(in_channels, out_channels))
        else:
            self.lins.append(nn.Linear(in_channels, hidden_channels))
            self.bns.append(nn.BatchNorm1d(hidden_channels))
            for _ in range(num_layers - 2):
                self.lins.append(nn.Linear(hidden_channels, hidden_channels))
                self.bns.append(nn.BatchNorm1d(hidden_channels))
            self.lins.append(nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for lin in self.lins:
            lin.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, b_data, input_tensor=False):
        if not input_tensor:
            x = b_data.x.shape[1]
        else:
            x = b_data
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = F.relu(x, inplace=True)
            x = self.bns[i](x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return x

In [6]:
class LINKX(nn.Module):	
    """ our LINKX method with skip connections 
        a = MLP_1(A), x = MLP_2(X), MLP_3(sigma(W_1[a, x] + a + x))
    """

    def __init__(self, in_channels, hidden_channels, out_channels, num_layers, num_nodes, dropout=.5, cache=False, inner_activation=False, inner_dropout=False, init_layers_A=1, init_layers_X=1):
        super(LINKX, self).__init__()	
        self.mlpA = MLP(num_nodes, hidden_channels, hidden_channels, init_layers_A, dropout=0)
        self.mlpX = MLP(in_channels, hidden_channels, hidden_channels, init_layers_X, dropout=0)
        self.W = nn.Linear(2*hidden_channels, hidden_channels)
        self.mlp_final = MLP(hidden_channels, hidden_channels, out_channels, num_layers, dropout=dropout)
        self.in_channels = in_channels
        self.num_nodes = num_nodes
        self.A = None
        self.inner_activation = inner_activation
        self.inner_dropout = inner_dropout

    def reset_parameters(self):	
        self.mlpA.reset_parameters()	
        self.mlpX.reset_parameters()
        self.W.reset_parameters()
        self.mlp_final.reset_parameters()	

    def forward(self, b_data):	
        
        m = b_data.num_nodes
        feat_dim = b_data.x.shape[1]
        row, col = b_data.edge_index
        
        row = row-row.min()
        A = SparseTensor(row=row, col=col,	
                 sparse_sizes=(m, self.num_nodes)
                        ).to_torch_sparse_coo_tensor()

        xA = self.mlpA(A, input_tensor=True)
        xX = self.mlpX(b_data.x, input_tensor=True)
        x = torch.cat((xA, xX), axis=-1)
        x = self.W(x)
        if self.inner_dropout:
            x = F.dropout(x)
        if self.inner_activation:
            x = F.relu(x)
        x = F.relu(x + xA + xX)
        x = self.mlp_final(x, input_tensor=True)

        return x

In [7]:
DATASET_NAME = 'karate'
data, dataset = get_data(DATASET_NAME, DIR=None, log=False, h_score=True, split_no=0); print("")

N  34  E  156  d  4.588235294117647 0.8020520210266113 0.7564102411270142 0.6170591711997986 -0.4756128787994385 


In [8]:
model  = LINKX(in_channels=data.x.shape[1], hidden_channels=32, out_channels=dataset.num_classes, num_layers=2, num_nodes = data.num_nodes)

In [9]:
model(data)

tensor([[-1.8602, -2.3305,  1.9020,  1.7530],
        [-0.0415, -0.8655,  0.3054, -1.3157],
        [ 0.9592, -0.1012,  0.1361, -0.2043],
        [ 0.1871,  0.2289, -0.2385, -0.5092],
        [-0.2558, -0.4815, -0.5436, -0.3338],
        [ 1.2181,  0.5065, -0.2723, -0.5030],
        [-1.4196, -1.9565, -1.3435,  0.5172],
        [-0.2967, -1.6790, -1.2853,  0.3231],
        [ 0.3240,  0.2654, -0.3580, -0.7607],
        [ 0.2989,  0.8665,  0.0217, -0.3542],
        [ 0.4382, -0.4370, -1.1081,  0.7084],
        [-0.6875,  0.6078,  0.0684,  0.2254],
        [ 0.4677,  0.3808, -0.2784,  0.1706],
        [-1.1885, -1.6461, -0.8335, -1.4923],
        [ 0.5087, -0.4047, -0.5542,  0.3314],
        [ 0.5350, -0.6899, -0.8723,  0.9766],
        [ 0.6135, -0.3675,  0.2930, -0.5557],
        [ 0.4203,  0.6643, -0.1018, -0.4959],
        [ 0.3971,  0.6886, -1.1383, -0.6583],
        [ 0.8777,  0.2066,  1.2963,  0.3344],
        [ 0.2656, -0.0056, -0.1563, -0.1857],
        [-0.1354, -0.4392,  0.1960